In [4]:
import pandas as pd
import numpy as np
import sklearn as sklearn
import seaborn as sb
from PIL import Image
from matplotlib import image
from matplotlib import pyplot
import glob

from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.neighbors import KNeighborsClassifier

from matplotlib import pyplot as plt



# Import Images from ORL dataset


In [5]:
def get_all_images(path):
    image_list = []
    for x in range(1,41): #loop on every subject folder
        for filename in glob.glob(path + 's' + str(x) + '/*.pgm'):              #get all images inside sX folder
            Im = Image.open(filename)
            image_list.append(Im)
    return image_list

# Data Setup


In [6]:
def generate_data_matrix(path):
  images = get_all_images(path)
  print(len(images)) #400

  #convert images list to a numpy array list
  images_vector = []
  for img in images:
      images_vector.append(np.array(img).ravel())

  print(len(images_vector)) #400
  print(images_vector[0].size) #10304
  print(images_vector[0].shape) #10304

  #df = pd.DataFrame(images_vector)
  y_labels = []
  for i in range(1,41):
      for j in range(1,11):
          y_labels.append(i)
  #df['label'] = pd.Series(y_labels)
  labels = np.asarray(y_labels , dtype = "int32")
  images_data = np.asarray(images_vector , dtype = "int32")
  return images_data , labels


# The ORL dataset PATH


In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
images_data , images_label = generate_data_matrix('/content/drive/MyDrive/ORL dataset/')

400
400
10304
(10304,)


# Split the data for training and testing sets


In [9]:
# testset
data_matrix_Test = []
label_matrix_Test = []

#trainigset
data_matrix_Train = []
label_matrix_Train = []

for i,index in zip(range(400) , range(400)):
  if(i % 2 == 0):
    data_matrix_Test.append(images_data[index])
    label_matrix_Test.append(images_label[index])
  else:
    data_matrix_Train.append(images_data[index])
    label_matrix_Train.append(images_label[index])




In [10]:
# the trainig_dataset and test_dataset
Data_matix_test = np.asarray(data_matrix_Test , dtype = "int32")
label_matix_test = np.asarray(label_matrix_Test , dtype = "int32")

Data_matix_train = np.asarray(data_matrix_Train , dtype = "int32")
label_matix_train = np.asarray(label_matrix_Train , dtype = "int32")
#print(label_matrix_Test)

print(Data_matix_test.shape)
print(Data_matix_train.shape)
print(label_matix_train.shape)
print(label_matix_test.shape)


(200, 10304)
(200, 10304)
(200,)
(200,)


# PCA

In [13]:
##Implementaion of Princible component analysis
alpha = np.array(
    [0.8,0.85,0.9,0.95]
    )

def PCA():
  test_matrix_mean = np.mean(Data_matix_test , axis = 0)                                #compute mean of each dim. in the data matrix 
  train_matrix_mean = np.mean(Data_matix_train , axis = 0)

  test_Z =  Data_matix_test - test_matrix_mean
  train_Z = Data_matix_train - train_matrix_mean
                                                                                        # center data step
  cov_train_matrix = np.cov(train_Z , rowvar = False)                                   # covariance matrix of centered data matrix Z
  eign_values , eign_vectors = np.linalg.eigh(cov_train_matrix)                         # compute the eign_val and eign_vectors of cov_matrix

  sorted_index = np.argsort(eign_values)[::-1]
  sorted_eign_values = eign_values[sorted_index]
  sorted_eign_vectors = eign_vectors[:,sorted_index]
  
  sum_eign_values_d = float(sum(sorted_eign_values))                                    #compute the sum of eign_values in d dim.
  
  reduced_dimention = 10304
  #for each alpha value
  for i in alpha:
    
    for itr in range(1 , 10304):

      eign_values_r = sorted_eign_values[:itr]                                     #subset of eign_values matrix to get the number of new dims.
      sum_eign_values_r = float(sum(eign_values_r))                                   #compute the sum of eign_Values in r dim matrix (subset)
      f_r = sum_eign_values_r / sum_eign_values_d                                     #f(r) = sum(r) / sum(d)
                                                                                      #check if the ratio is one of given alpha in pdf
    
      if(f_r >= i):
        reduced_dimention = itr                                                       #if any one is ture break
        break
  
    projection_matrix = sorted_eign_vectors[:,0:reduced_dimention]

    reduced_data_train_matrix = np.dot(projection_matrix.transpose() , train_Z.transpose())
    reduced_data_test_matrix = np.dot(projection_matrix.transpose() , test_Z.transpose())

    print("For alpha: " + str(i))

    KNN_Imp(reduced_data_train_matrix,label_matix_train , reduced_data_test_matrix,label_matix_test)







   # KNN


In [12]:
def KNN_Imp(Data_train,Label_train,Data_test,Label_test):
  neighbours = [1,3,5,7]
  accuracy_matrix = []
  
  for i,neighbour in zip(range(len(neighbours)),neighbours):
      print("Neig = " + "[" + str(neighbour) + "]")
      
      classification_KNN = KNeighborsClassifier(n_neighbors = neighbour) 
      classification_KNN.fit(Data_train.transpose(), Label_train) 
      output = classification_KNN.predict(Data_test.transpose())
      accuracy_matrix.append(accuracy_score(output,Label_test)) 
     
      print("Accuracy_Sc is: " + str(accuracy_matrix[i]))
     
      count = 0
      for i in range(len(output)):          
          if((output[i]) != (Label_test[i])):
              count+=1
      
      print("Numbers_of_wrong_classifications=  " + str(count) + "\n")
      #plt.plot(accuracy_matrix,neighbours)
      #plt.show()


In [14]:
PCA()

For alpha: 0.8
Neig = [1]
Accuracy_Sc is: 0.92
Numbers_of_wrong_classifications=  16

Neig = [3]
Accuracy_Sc is: 0.83
Numbers_of_wrong_classifications=  34

Neig = [5]
Accuracy_Sc is: 0.77
Numbers_of_wrong_classifications=  46

Neig = [7]
Accuracy_Sc is: 0.725
Numbers_of_wrong_classifications=  55

For alpha: 0.85
Neig = [1]
Accuracy_Sc is: 0.92
Numbers_of_wrong_classifications=  16

Neig = [3]
Accuracy_Sc is: 0.84
Numbers_of_wrong_classifications=  32

Neig = [5]
Accuracy_Sc is: 0.77
Numbers_of_wrong_classifications=  46

Neig = [7]
Accuracy_Sc is: 0.715
Numbers_of_wrong_classifications=  57

For alpha: 0.9
Neig = [1]
Accuracy_Sc is: 0.915
Numbers_of_wrong_classifications=  17

Neig = [3]
Accuracy_Sc is: 0.825
Numbers_of_wrong_classifications=  35

Neig = [5]
Accuracy_Sc is: 0.765
Numbers_of_wrong_classifications=  47

Neig = [7]
Accuracy_Sc is: 0.715
Numbers_of_wrong_classifications=  57

For alpha: 0.95
Neig = [1]
Accuracy_Sc is: 0.92
Numbers_of_wrong_classifications=  16

Neig = [3